In [1]:
%matplotlib inline

# Optimización de los parámetros del modelo

Ahora que tenemos un modelo y datos, es hora de entrenar, validar y probar nuestro modelo optimizando sus parámetros en nuestros datos. Entrenar un modelo es un proceso iterativo; en cada iteración (llamada época) el modelo hace una suposición sobre la salida, calcula el error en su suposición (pérdida), recopila las derivadas del error con respecto a sus parámetros (como vimos en la sección anterior) y **optimiza** estos parámetros usando el descenso de gradiente. Para obtener un recorrido más detallado de este proceso, consulte este video sobre [backpropagation de 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8).

## Código previo necesario

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## Hiperparámetros

Los hiperparámetros son parámetros ajustables que le permiten controlar el proceso de optimización del modelo. Diferentes valores de hiperparámetros pueden afectar el entrenamiento del modelo y las tasas de convergencia (lea más sobre el ajuste de hiperparámetros)

Definimos los siguientes hiperparámetros para el entrenamiento:
 * **Número de épocas**: el número de veces que se itera sobre el conjunto de datos
 * **Tamaño de lote**: el número de muestras de datos propagadas a través de la red antes de que se actualicen los parámetros.
 * **Tasa de aprendizaje**: cuánto actualizar los parámetros de los modelos en cada lote/época. Los valores más pequeños producen una velocidad de aprendizaje lenta, mientras que los valores altos pueden provocar un comportamiento impredecible durante el entrenamiento.

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Bucle de optimización

Una vez que establezcamos nuestros hiperparámetros, podemos entrenar y optimizar nuestro modelo con un ciclo de optimización. Cada iteración del ciclo de optimización se denomina época.

Cada época consta de dos partes principales:
 * **El ciclo de entrenamiento**: itera sobre el conjunto de datos de entrenamiento e intenta converger a los parámetros óptimos.
 * **El ciclo de validación/prueba**: itera sobre el conjunto de datos de prueba para comprobar si el rendimiento del modelo está mejorando.

Familiaricémonos brevemente con algunos de los conceptos utilizados en el ciclo de entrenamiento.

### Loss Function

Cuando se le presentan algunos datos de entrenamiento, es probable que nuestra red no capacitada no dé la respuesta correcta. La función de pérdida mide el grado de disimilitud del resultado obtenido con el valor objetivo, y es la función de pérdida lo que queremos minimizar durante el entrenamiento. Para calcular la pérdida, hacemos una predicción utilizando las entradas de nuestra muestra de datos dada y la comparamos con el valor real de la etiqueta de datos.

Las funciones de pérdida comunes incluyen [``nn.MSELoss``](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss) (error cuadrático medio) para tareas de regresión y [``nn.NLLLoss``](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss) (probabilidad de registro negativo) para clasificación. [``nn.CrossEntropyLoss``](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) combina ``nn.LogSoftmaxy`` ``nn.NLLLoss``.

Pasamos los logits de salida de nuestro modelo a ``nn.CrossEntropyLoss``, que normalizarán los logits y calcularán el error de predicción.

In [5]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### Optimizador

La optimización es el proceso de ajustar los parámetros del modelo para reducir el error del modelo en cada paso de entrenamiento. **Los algoritmos de optimización** definen cómo se realiza este proceso (en este ejemplo usamos Descenso de gradiente estocástico). Toda la lógica de optimización está encapsulada en el objeto ``optimizer``. Aquí usamos el optimizador SGD; Además, hay muchos [optimizadores diferentes](https://pytorch.org/docs/stable/optim.html) disponibles en PyTorch, como ADAM y RMSProp, que funcionan mejor para diferentes tipos de modelos y datos.

Inicializamos el optimizador registrando los parámetros del modelo que necesitan ser entrenados y pasando el hiperparámetro de tasa de aprendizaje.

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Dentro del ciclo de entrenamiento, la optimización ocurre en tres pasos:
 * Llamada a ``optimizer.zero_grad()`` para restablecer los gradientes de los parámetros del modelo. Los degradados se suman por defecto; para evitar el doble conteo, los ponemos a cero explícitamente en cada iteración.
 * Retropropagación de la pérdida de predicción con una llamada a ``loss.backwards()``. PyTorch deposita los gradientes de pérdida con cada parámetro.
 * Una vez que tenemos nuestros degradados, llamamos ``optimizer.step()`` a ajustar los parámetros por los degradados recogidos en la backward.

### Implementación completa

Definimos ``train_loop`` que recorre nuestro código de optimización y ``test_loop`` que evalúa el rendimiento del modelo en comparación con nuestros datos de prueba.

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Inicializamos la función de pérdida y el optimizador, y lo pasamos a ``train_loop`` y ``test_loop``. No dude en aumentar el número de épocas para realizar un seguimiento de la mejora del rendimiento del modelo.

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.308122  [    0/60000]
loss: 2.294896  [ 6400/60000]
loss: 2.277630  [12800/60000]
loss: 2.264670  [19200/60000]
loss: 2.258785  [25600/60000]
loss: 2.227958  [32000/60000]
loss: 2.232154  [38400/60000]
loss: 2.202238  [44800/60000]
loss: 2.206822  [51200/60000]
loss: 2.160461  [57600/60000]
Test Error: 
 Accuracy: 50.9%, Avg loss: 2.164159 

Epoch 2
-------------------------------
loss: 2.179595  [    0/60000]
loss: 2.167129  [ 6400/60000]
loss: 2.108799  [12800/60000]
loss: 2.117221  [19200/60000]
loss: 2.077025  [25600/60000]
loss: 2.017354  [32000/60000]
loss: 2.043247  [38400/60000]
loss: 1.965428  [44800/60000]
loss: 1.979008  [51200/60000]
loss: 1.892849  [57600/60000]
Test Error: 
 Accuracy: 55.9%, Avg loss: 1.896715 

Epoch 3
-------------------------------
loss: 1.932781  [    0/60000]
loss: 1.901781  [ 6400/60000]
loss: 1.777608  [12800/60000]
loss: 1.815290  [19200/60000]
loss: 1.720087  [25600/60000]
loss: 1.661584  [32000/600